In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import time

In [2]:
def parse_banco_chile(url, url_desc):

    # create a new Chrome browser instance
    driver = webdriver.Firefox()

    # navigate to the website you want to scrape
    driver.get(url)

    # wait for the JavaScript to generate the content
    time.sleep(3)
    driver.implicitly_wait(30)

    # parse the HTML content of the page
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # close the browser instance
    driver.quit()

    # find all elements with the specified class
    items = soup.find_all('div', class_='col-12 col-sm-6 col-md-3 mb-4 new-beneficios-content-item')

    df = []

    # extract the information you need from each item
    for item in items:
        # do your scraping here, for example:
        link = item.find('a')['href']
        id = item.find('a')['id']
        desc1 = item.find('strong').text
        try:
            desc2 = item.find('span').text
        except:
            desc2 = 'N_A'

        df.append([id, link, desc1, desc2])

    df = pd.DataFrame(df, columns=['ID', 'LINK', 'DESCRICION_1', 'DESCRIPCION_2'])

    df.insert(loc=0, column='ENTIDAD', value='BANCO DE CHILE')
    df.insert(loc=1, column='DESCRIPCION_URL', value=url_desc)

    return df


In [3]:
df1 = parse_banco_chile('https://portales.bancochile.cl/personas/beneficios/?categoria=beneficios+del+mes', 'BENEFICIOS DEL MES')
df2 = parse_banco_chile('https://portales.bancochile.cl/personas/beneficios/?categoria=marcas', 'MARCAS')
df3 = parse_banco_chile('https://portales.bancochile.cl/personas/beneficios/?categoria=restaurantes', 'RESTAURANTES')
df4 = parse_banco_chile('https://portales.bancochile.cl/personas/beneficios/?categoria=delivery', 'DELIVERY')
df5 = parse_banco_chile('https://portales.bancochile.cl/personas/beneficios/?categoria=panoramas', 'PANORAMAS')

df_final = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [4]:
df_final['']

,ENTIDAD,DESCRIPCION_URL,ID,LINK,DESCRICION_1,DESCRIPCION_2
0,BANCO DE CHILE,BENEFICIOS DEL MES,ppp_beneficio-link-back-to-school,https://portales.bancochile.cl/personas/campan...,Vuelve a clases con el Chile,Los mejores descuentos y beneficios son con tu...
1,BANCO DE CHILE,BENEFICIOS DEL MES,ppp_beneficio-link-big5-2023,https://portales.bancochile.cl/personas/campan...,Date un gusto todos los días,Hasta 40% dcto. pagando con tus Tarjetas del C...
2,BANCO DE CHILE,BENEFICIOS DEL MES,ppp_beneficio-link-30-dcto-restaurant,https://portales.bancochile.cl/personas/campan...,Jueves de terraza,30% dcto. en restaurantes pagando con tus Tarj...
3,BANCO DE CHILE,BENEFICIOS DEL MES,ppp_beneficio-link-fan-clan-beneficios,https://portales.bancochile.cl/personas/campan...,Beneficios FAN CLAN,Por ser parte de FAN Clan tienes estos exclusi...
4,BANCO DE CHILE,BENEFICIOS DEL MES,ppp_beneficio-link-pet-family,https://portales.bancochile.cl/personas/benefi...,PET FAMILY,20% dcto. Todos los días
...,...,...,...,...,...,...
452,BANCO DE CHILE,PANORAMAS,ppp_beneficio-link-piknic-6-25-de-febrero,https://portales.bancochile.cl/personas/benefi...,Piknic #6 – 25 de febrero,Compra tus entradas con los mejores beneficios.
453,BANCO DE CHILE,PANORAMAS,ppp_beneficio-link-ludovico-einaudi-22-de-marz...,https://portales.bancochile.cl/personas/benefi...,Ludovico Einaudi – 22 de marzo en el Teatro Mu...,Compra tus entradas con los mejores beneficios.
454,BANCO DE CHILE,PANORAMAS,ppp_beneficio-link-travel-duty,https://portales.bancochile.cl/personas/benefi...,Travel Duty,Canjea tus Dólares-Premio por los mejores prod...
455,BANCO DE CHILE,PANORAMAS,ppp_beneficio-link-travel-tienda,https://portales.bancochile.cl/personas/benefi...,Travel Tienda,Encuentra las mejores ofertas y productos y co...


In [61]:
df_final['DESCRIPCION_3'] = pd.NA
df_final['CONDICIONES'] = pd.NA

for ix in df_final.index:
    url_req = df_final.at[ix, 'LINK']
    page = requests.get(url_req)

    soup = BeautifulSoup(page.content, 'html.parser')
    try:
        filtrado = soup.find_all('div', class_='detail__description')[0]['v-html']
        soup = BeautifulSoup(filtrado, 'html.parser')
        desc3 = soup.find('p').text
        cond = ''
        for c in soup.find_all('li'):
            cond += c.text + ';'
        cond = cond[:-1]
        
        df_final.at[ix, 'DESCRIPCION_3'] = desc3
        df_final.at[ix, 'CONDICIONES'] = cond
    except:
        pass

In [62]:
df_final

,ENTIDAD,DESCRIPCION_URL,ID,LINK,DESCRICION_1,DESCRIPCION_2,DESCRIPCION_3,CONDICIONES
0,BANCO DE CHILE,BENEFICIOS DEL MES,ppp_beneficio-link-back-to-school,https://portales.bancochile.cl/personas/campan...,Vuelve a clases con el Chile,Los mejores descuentos y beneficios son con tu...,<NA>,<NA>
1,BANCO DE CHILE,BENEFICIOS DEL MES,ppp_beneficio-link-big5-2023,https://portales.bancochile.cl/personas/campan...,Date un gusto todos los días,Hasta 40% dcto. pagando con tus Tarjetas del C...,<NA>,<NA>
2,BANCO DE CHILE,BENEFICIOS DEL MES,ppp_beneficio-link-30-dcto-restaurant,https://portales.bancochile.cl/personas/campan...,Jueves de terraza,30% dcto. en restaurantes pagando con tus Tarj...,<NA>,<NA>
3,BANCO DE CHILE,BENEFICIOS DEL MES,ppp_beneficio-link-fan-clan-beneficios,https://portales.bancochile.cl/personas/campan...,Beneficios FAN CLAN,Por ser parte de FAN Clan tienes estos exclusi...,<NA>,<NA>
4,BANCO DE CHILE,BENEFICIOS DEL MES,ppp_beneficio-link-pet-family,https://portales.bancochile.cl/personas/benefi...,PET FAMILY,20% dcto. Todos los días,PetFamily es el ecosistema más grande de todo ...,15% de descuento en toda la academia. Cursos g...
...,...,...,...,...,...,...,...,...
452,BANCO DE CHILE,PANORAMAS,ppp_beneficio-link-piknic-6-25-de-febrero,https://portales.bancochile.cl/personas/benefi...,Piknic #6 – 25 de febrero,Compra tus entradas con los mejores beneficios.,NaN,NaN
453,BANCO DE CHILE,PANORAMAS,ppp_beneficio-link-ludovico-einaudi-22-de-marz...,https://portales.bancochile.cl/personas/benefi...,Ludovico Einaudi – 22 de marzo en el Teatro Mu...,Compra tus entradas con los mejores beneficios.,NaN,NaN
454,BANCO DE CHILE,PANORAMAS,ppp_beneficio-link-travel-duty,https://portales.bancochile.cl/personas/benefi...,Travel Duty,Canjea tus Dólares-Premio por los mejores prod...,NaN,NaN
455,BANCO DE CHILE,PANORAMAS,ppp_beneficio-link-travel-tienda,https://portales.bancochile.cl/personas/benefi...,Travel Tienda,Encuentra las mejores ofertas y productos y co...,NaN,NaN


In [64]:
df_final.to_csv('banco_chile.csv', sep=';', decimal=',', index=False, encoding='utf-8-sig')